<a href="https://colab.research.google.com/github/kunai-3txk/colab/blob/main/20221010_TOKUHO_simu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [27]:
#テストデータの作成
# Y: weight X:sex old Intervention

#1000名に対して100名介入とする
N = 1000 #総数
N_intervention = 100 #介入数

old = np.random.randint(30,60,N) # 年齢は30-59の一様分布とする
sex = np.random.randint(0,2,N) # 性別は0:男性 1:女性 分布はランダム
weight = np.random.normal(60,10,N) #体重は平均60偏差10の正規分布

df_zeros = np.zeros(N-N_intervention)
df_ones = np.ones(N_intervention)
intervention = np.hstack((df_zeros,df_ones)) #介入フラグ

#display(old)
#display(sex)
#display(weight)
#display(intervention)

ar = np.vstack((old,sex,weight,intervention))
columns=['old','sex','weight','intervention']

df = pd.DataFrame(ar.T,columns=columns)

display(df)

,old,sex,weight,intervention
0,57.0,0.0,51.843929,0.0
1,38.0,0.0,44.395882,0.0
2,57.0,1.0,55.997790,0.0
3,40.0,1.0,53.665595,0.0
4,37.0,1.0,61.171136,0.0
...,...,...,...,...
995,57.0,1.0,57.384729,1.0
996,46.0,1.0,62.965203,1.0
997,34.0,0.0,66.059994,1.0
998,37.0,0.0,51.292803,1.0


In [ ]:
#介入効果の作成
#体重が多いほど効果が高いとみなし…　⇒ まずは単純モデルの線形で
# 効果（体重減少） = 5.0 X + 誤差項(平均1偏差0.5の正規分布)

df_e_term = pd.DataFrame(np.random.normal(1,0.5,N),columns=['e'])

df['weight'] = df['weight'] - (df['intervention']*5.0 + df_e_term['e'])

#display(df)

In [29]:
#ざっくり状況確認


#ATE 算出
#介入群(T=1)
df_t1 = df[df['intervention']==1]
#対照群(T=0)
df_t0 = df[df['intervention']==0]

diff = df_t1['weight'].mean()-df_t0['weight'].mean()

display(diff) #-4.521749080827185

-4.521749080827185

In [36]:
#-----------------------
# S-Learner 構築
# https://zenn.dev/s1ok69oo/articles/491fbaf2092cd5
#-----------------------

# 手順1: モデルを学習
# S-Learnerなので単独のモデルで実装
reg_s1 = RandomForestRegressor(max_depth=3, random_state=0)
X = df.loc[:,['old','sex','intervention']]
reg_s1.fit(X, df['weight'])

# 手順2: 広告を見ていない(T=0)データと広告を見た(T=1)データを作成
X_0 = X.copy()
X_0['intervention'] = 0

X_1 = X.copy()
X_1['intervention'] = 1

# 手順3: 効果の推定
mu_0 = reg_s1.predict(X_0)
mu_1 = reg_s1.predict(X_1)
display((mu_1 - mu_0).mean()) #-4.566057331823679

-4.566057331823679

In [38]:
#-----------------------
# T-Learner 構築
# https://zenn.dev/s1ok69oo/articles/4a36fee0297234
#-----------------------

# 手順1: 広告を見ていないグループの回帰モデルを作成
reg_t1 = RandomForestRegressor(max_depth=3, random_state=0)
reg_t1.fit(df_t0.loc[:,['old','sex','intervention']], df_t0['weight'])

# 手順2: 広告を見たグループの回帰モデルを作成
reg_t2 = RandomForestRegressor(max_depth=3, random_state=0)
reg_t2.fit(df_t1.loc[:,['old','sex','intervention']], df_t1['weight'])

# 手順3: 効果の推定
mu_0 = reg_t1.predict(X)
mu_1 = reg_t2.predict(X)
display((mu_1 - mu_0).mean()) #-4.479550639531771

-4.479550639531771